In [1]:
import numpy as np
from connection_complexity.data.raw_data.EDF.edf_helpers import read_edf
from tqdm import tqdm
import os
from mne import make_fixed_length_epochs


file = "/media/dan/Data/data/iEEG/raw_ieeg/baseline_patients/baseline_edfs/034_Baseline.EDF"
output_path = "/home/dan/data/connectivity/pyspi_testing/nonoverlappingepochs_data"
# make output path
if not os.path.exists(output_path):
    os.makedirs(output_path)

project_path = "/home/dan/data/connectivity/pyspi_testing"


raw = read_edf(file, preload=True)

# filter
sfreq = raw.info["sfreq"]
nyq_freq = raw.info["sfreq"] // 2 - 1
line_freq = 60
l_freq = 0.5
h_freq = min(nyq_freq, 300)
raw = raw.filter(l_freq=l_freq, h_freq=h_freq)
freqs = np.arange(line_freq, max(h_freq, nyq_freq), line_freq)
raw = raw.notch_filter(freqs=freqs, method="fir")

# average reference
raw = raw.set_eeg_reference(ref_channels="average", projection=False, verbose=False)

data = raw.get_data()
maxidx = data.shape[1]
digit_count = len(str(abs(maxidx))) 

# non overlapping epochs
dur_sec = .5
overlap_sec = 0
epochs = make_fixed_length_epochs(raw, duration=dur_sec,overlap=overlap_sec, preload=True)
epochs = epochs.get_data()


for i, e in enumerate(tqdm(epochs)): # start 1 minute in only use first 5 seconds
    start_idx = int(sfreq * dur_sec) * i
    np.save(os.path.join(output_path, f"034_epoch_{start_idx:0{digit_count}}.npy"), e)

# overlapping epochs with overlap of 1 sample
# start = int(sfreq * 60) # start 1 minute in
# for i in tqdm(range(start, start+int(sfreq * 1))): # start 1 minute in only use first 5 seconds
#     np.save(os.path.join(output_path, f"034_epoch_{i}.npy"), data[:,i:i+int(0.5*sfreq)])

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 3e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 300.00 Hz
- Upper transition bandwidth: 75.00 Hz (-6 dB cutoff frequency: 337.50 Hz)
- Filter length: 13517 samples (6.600 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.0s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 13517 samples (6.600 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.9s


Not setting metadata
606 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 606 events and 1024 original time points ...
0 bad epochs dropped


100%|██████████| 606/606 [00:00<00:00, 1216.27it/s]


In [9]:
# define the YAML file location
yaml_file = os.path.join(output_path,"run_config.yaml")

# remove the file if it already exists
if os.path.exists(yaml_file):
    os.remove(yaml_file)

# ps -> rows = processes; columns = time pts.
dim_order = "ps"

for f in sorted(os.listdir(output_path)): # start 1 minute in only use first 5 seconds
    # get the epoch number
    i = f.split("_")[-1].split(".")[0]
    # define template string and fill in variables
    lbl = f"034_epoch_{i}"
    path = os.path.join(output_path, lbl)
    yaml_string = "{{file: {file}.npy, name: {key}, dim_order: {dim_order}, labels: [{key}]}}\n"
    yaml_string_formatted = f"- {yaml_string.format(file=path, key=lbl, dim_order=dim_order)}"

    # append line to file
    with open(yaml_file, "a") as f:
        f.write(yaml_string_formatted)

In [ ]:
# import yaml
# doc = "/home/dan/data/connectivity/pyspi_testing/nonoverlappingepochs_data/run_config.yaml"
# with open(doc) as d:
#     yf = yaml.load(d,Loader=yaml.FullLoader)